## 安装PaddleSeg以及更新依赖

In [ ]:
#!git clone https://gitee.com/paddlepaddle/PaddleSeg.git

In [ ]:
!pip install paddlepaddle-gpu==2.1.1.post101 -f https://paddlepaddle.org.cn/whl/mkl/stable.html

In [ ]:
!pip install -r PaddleSeg/requirements.txt

## 解压数据集

In [2]:
!unzip data/data77571/train_and_label.zip -d data/

In [3]:
!unzip data/data77571/img_test.zip -d data/

## 生成训练列表文件

In [ ]:
import os
img_train_dir = os.listdir('./data/img_train')
train_val_ratio = 0.9 #划分训练集和验证集的比例
train_img_num = int(len(img_train_dir)*train_val_ratio)
val_img_num = len(img_train_dir) - train_img_num
train_list_origin = img_train_dir[:train_img_num]
val_list_origin = img_train_dir[train_img_num:]
print(len(train_list_origin),len(val_list_origin)) #输出训练集和验证集数量

#生成trainlist
with open('train_list.txt','w') as f:
    for i in range(len(train_list_origin)):
        train_img_name = 'img_train/' + train_list_origin[i]
        train_lab_name = 'lab_train/' + train_list_origin[i].split('.')[0] + '.png'
        f.write(train_img_name + ' ' + train_lab_name + '\n')
#生成vallist
with open('val_list.txt','w') as f:
    for i in range(len(val_list_origin)):
        val_img_name = 'img_train/' + val_list_origin[i]
        val_lab_name = 'lab_train/' + val_list_origin[i].split('.')[0] + '.png'
        f.write(val_img_name + ' ' + val_lab_name + '\n')

## 配置训练参数
#### 在根目录下目录下创建一个命名为swin.yml的配置文件，将以下参数复制到文件即可

```
batch_size: 8
iters: 200000

model:
  type: MLATransformer
  backbone:
    type: SwinTransformer_base_patch4_window7_224
    ape: False
    drop_path_rate: 0.3
    patch_norm: True
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/swin_transformer_base_patch4_window7_224_imagenet_1k.tar.gz
  num_classes: 4
  in_channels: [128, 256, 512, 1024]
  mlahead_channels: 128

train_dataset:
  type: Dataset
  dataset_root: ./data
  train_path: train_list.txt
  num_classes: 4
  edge: True
  transforms:
    - type: RandomHorizontalFlip
    - type: RandomVerticalFlip
    - type: RandomRotation
      max_rotation: 180
      im_padding_value: [127.5, 127.5, 127.5]
      label_padding_value: 255
    - type: RandomBlur
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Resize
      target_size: [224, 224]
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: ./data
  val_path: val_list.txt
  num_classes: 4
  transforms:
    - type: Resize
      target_size: [224, 224]
    - type: Normalize
  mode: val

optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

learning_rate:
  value: 0.003
  decay:
    type: poly
    power: 0.9
    end_lr: 0.00001

loss:
  types:
    - type: CrossEntropyLoss
    - type: CrossEntropyLoss
  coef: [1, 0.4]

```

## 调优思路解析

> 1.尝试使用了SwinTransformer网络模型（本次就是想试试transformer的水）  
2.在数据增强方面，使用了水平旋转、垂直旋转和常规的明亮度、对比度和饱和度调节

## 开始训练

In [ ]:
!python PaddleSeg/train.py \
       --config swin.yml \
       --do_eval \
       --use_vdl \
       --save_interval 1000 \
       --save_dir output

In [ ]:
!python PaddleSeg/train.py \
       --config swin.yml \
       --do_eval \
       --use_vdl \
       --save_interval 750 \
       --save_dir output

## 预测图片

> 注意：因为Predict.py文件默认是输出渲染后的彩色图片，但比赛需要提交原图，因此我们需要修改一下PaddleSeg/paddleseg/utils/visualize.py文件

> 修改方法是把第60和61行注释掉，让pseudo_color_prediction文件夹直接输出原图，然后下载再上传系统评估即可  
    #color_map = get_color_map_list(256)  
    #pred_mask.putpalette(color_map)

In [4]:
!python PaddleSeg/predict.py \
       --config swin.yml \
       --model_path output/best_model/model.pdparams \
       --image_path data/img_testA \
       --save_dir ./result